In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold
import winsound as sd

C:\Users\KimHojin\anaconda3\envs\py310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 지정된 프로시저를 찾을 수 없습니다'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
from PIL import Image
import numpy as np
dataset=[]
#train_dataset=[]
#test_dataset=[]
for n in range(1,141):
  try:
    image = Image.open('Dataset/o_{}.png'.format(n)).convert('L')
  except:
    image = Image.open('Dataset/o_{}.jpg'.format(n)).convert('L')
  image = np.array(image)
  image =torch.FloatTensor([image])
  #data=image,torch.FloatTensor([[0,1]])
  data=image,0
  dataset.append(data)
  try:
    image = Image.open('Dataset/x_{}.png'.format(n)).convert('L')
  except:
    image = Image.open('Dataset/x_{}.jpg'.format(n)).convert('L')
  image = np.array(image)
  image=torch.FloatTensor([image])
  #data=image,torch.FloatTensor([[1,0]])
  data=image,1
  dataset.append(data)

C:\Users\KimHojin\AppData\Local\Temp\ipykernel_25520\2429053110.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_8f7uhuge1i\croot\pytorch-select_1717607507421\work\torch\csrc\utils\tensor_new.cpp:277.)
  image =torch.FloatTensor([image])


In [10]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(32, 64, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(64, 128, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(128, 256, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 256, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)


if __name__ == '__main__':

  # Configuration options
  k_folds = 7
  num_epochs = 10
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler)
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=65536, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch    10: 18.396
Loss after mini-batch    20: 10.598
Loss after mini-batch    30: 6.991
Loss after mini-batch    40: 6.418
Loss after mini-batch    50: 6.803
Loss after mini-batch    60: 5.048
Loss after mini-batch    70: 4.240
Loss after mini-batch    80: 3.627
Loss 

In [4]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 10
  num_epochs = 10
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch     1: 0.950
Loss after mini-batch     2: 0.646
Loss after mini-batch     3: 0.635
Loss after mini-batch     4: 0.701
Loss after mini-batch     5: 0.706
Loss after mini-batch     6: 0.583
Loss after mini-batch     7: 1.034
Loss after mini-batch     8: 0.783
Loss after

In [6]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 10
  num_epochs = 10
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch     1: 1.022
Loss after mini-batch     2: 0.679
Loss after mini-batch     3: 0.712
Loss after mini-batch     4: 0.667
Loss after mini-batch     5: 0.662
Loss after mini-batch     6: 0.737
Loss after mini-batch     7: 0.510
Loss after mini-batch     8: 0.552
Loss after

In [7]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 10
  num_epochs = 100
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch     1: 0.848
Loss after mini-batch     2: 0.705
Loss after mini-batch     3: 0.730
Loss after mini-batch     4: 0.929
Loss after mini-batch     5: 0.682
Loss after mini-batch     6: 0.845
Loss after mini-batch     7: 0.782
Loss after mini-batch     8: 0.589
Loss after

PermissionError: [WinError 5] 액세스가 거부되었습니다

In [4]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 20, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(40, 60, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(60, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 10
  num_epochs = 20
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=20, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch%10 == 9:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0 and epoch % 10 == 9:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 60, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(60, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 10
Loss after mini-batch     1: 0.097
Loss after mini-batch     2: 0.084
Loss after mini-batch     3: 0.076
Loss after mini-batch     4: 0.070
Loss after mini-batch     5: 0.194
Loss after mini-batch     6: 0.014
Loss after mini-batch     7: 0.029
Loss after mini-batch     8: 0.084
Loss afte

In [5]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 30
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch     1: 0.784
Loss after mini-batch     2: 0.672
Loss after mini-batch     3: 0.675
Loss after mini-batch     4: 0.744
Loss after mini-batch     5: 0.653
Loss after mini-batch     6: 0.821
Loss after mini-batch     7: 0.847
Loss after mini-batch     8: 0.931
Loss after

In [6]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 10
  num_epochs = 20
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=25, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=25, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 1 == 0:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 1
Loss after mini-batch     1: 0.877
Loss after mini-batch     2: 0.714
Loss after mini-batch     3: 0.904
Loss after mini-batch     4: 0.704
Loss after mini-batch     5: 0.719
Loss after mini-batch     6: 0.857
Loss after mini-batch     7: 0.634
Loss after mini-batch     8: 0.641
Loss after

In [9]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 40
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch % 10 ==9:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9 and epoch % 10 ==9:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 10
Loss after mini-batch    10: 0.000
Loss after mini-batch    20: 0.002
Loss after mini-batch    30: 0.045
Loss after mini-batch    40: 0.005
Loss after mini-batch    50: 0.069
Loss after mini-batch    60: 0.013
Loss after mini-batch    70: 0.013
Loss after mini-batch    80: 0.000
Loss afte

In [10]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 18
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch % 3 ==2:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9 and epoch % 3 == 2:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 3
Loss after mini-batch    10: 0.225
Loss after mini-batch    20: 0.077
Loss after mini-batch    30: 2.602
Loss after mini-batch    40: 2.168
Loss after mini-batch    50: 0.330
Loss after mini-batch    60: 0.875
Loss after mini-batch    70: 1.741
Loss after mini-batch    80: 2.421
Loss after

In [11]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 18
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=15, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=15, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch % 3 ==2:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9 and epoch % 3 == 2:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 3
Loss after mini-batch    10: 3.132
Starting epoch 6
Loss after mini-batch    10: 1.293
Starting epoch 9
Loss after mini-batch    10: 0.323
Starting epoch 12
Loss after mini-batch    10: 0.147
Starting epoch 15
Loss after mini-batch    10: 0.030
Starting epoch 18
Loss after mini-batch    10

In [12]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 50),
      nn.ReLU(),
      nn.Linear(50, 20),
      nn.ReLU(),
      nn.Linear(20, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 27
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=15, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=15, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch % 3 ==2:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9 and epoch % 3 == 2:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=50, bias=True)
Reset trainable parameters of layer = Linear(in_features=50, out_features=20, bias=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=2, bias=True)
Starting epoch 3
Loss after mini-batch    10: 4.158
Starting epoch 6
Loss after mini-batch    10: 1.760
Starting epoch 9
Loss after mini-batch    10: 0.366
Starting epoch 12
Loss after mini-batch    10: 0.120
Starting epoch 15
Loss after mini-batch    10: 0.079
Starting epoch 18
Loss after mini-batch    10

In [13]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()
class SimpleConvNet(nn.Module):
  '''
    Simple Convolutional Neural Network
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Conv2d(1, 10, kernel_size=3), # 1*300*300 -> 10*298*298
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #10*298*298 -> 10*149*149
      #풀링 추가
      nn.Conv2d(10, 20, kernel_size=3), #10*149*149 -> 20*147*147
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #20*147*147 -> 20*73*73
      #풀링 추가
      nn.Conv2d(20, 40, kernel_size=3), #20*73*73->40*71*71
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #40*71*71->40*35,35
      #풀링 추가
      nn.Conv2d(40, 80, kernel_size=3), #40*35*35 -> 80*33*33
      nn.ReLU(),
      nn.MaxPool2d((2,2)), #80*33*33->80*16*16
      #풀링 추가
      nn.Flatten(),
      nn.Linear(16 * 16 * 80, 200),
      nn.ReLU(),
      nn.Linear(200, 200),
      nn.ReLU(),
      nn.Linear(200, 2)
    )

  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)
if __name__ == '__main__':

  # Configuration options
  k_folds = 5
  num_epochs = 15
  loss_function = nn.CrossEntropyLoss()

  # For fold results
  results = {}

  # Set fixed random number seed
  torch.manual_seed(42)

  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # Start print
  print('--------------------------------')

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=train_subsampler,num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=1, sampler=test_subsampler,num_workers=4)

    # Init the neural network
    network = SimpleConvNet()
    network.apply(reset_weights)

    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      if epoch % 3 ==2:
        print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0.0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):

        # Get inputs
        inputs, targets = data
        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = network(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
        if i % 10 == 9 and epoch % 3 == 2:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 1))
            current_loss = 0.0

    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')

    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)

        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

      # Print accuracy
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      print('--------------------------------')
      results[fold] = 100.0 * (correct / total)

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()


--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1))
Reset trainable parameters of layer = Linear(in_features=20480, out_features=200, bias=True)
Reset trainable parameters of layer = Linear(in_features=200, out_features=200, bias=True)
Reset trainable parameters of layer = Linear(in_features=200, out_features=2, bias=True)
Starting epoch 3
Loss after mini-batch    10: 1.103
Loss after mini-batch    20: 5.597
Loss after mini-batch    30: 2.717
Loss after mini-batch    40: 0.773
Loss after mini-batch    50: 0.859
Loss after mini-batch    60: 2.168
Loss after mini-batch    70: 1.998
Loss after mini-batch    80: 1.618
Loss a